<a href="https://colab.research.google.com/github/Maximi652/efficient-slm-architectures/blob/main/efficient_slm_architectures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio, torch, transformers

In [ ]:
import json

# Open the file in read mode ('r')
with open("/Training_Data/12B_combined_golden.json", 'r') as f:
    # Now you can load the JSON data from the file object 'f'
    training_data = json.load(f)

# print(json.dumps(training_data['questions'][0], indent=2))

In [11]:
import gradio as gr
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-7B-Instruct"

# Tokenizer und Modell laden
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True, device_map="auto").eval()

# Chatfunktion
def generate_response(message, history):
    # Geschichte in Chatformat aufbauen (für Prompt Template)
    history = history or []
    messages = [{"role": "system", "content": "You are a helpful assistant."}]
    for user, assistant in history:
        messages.append({"role": "user", "content": user})
        messages.append({"role": "assistant", "content": assistant})
    messages.append({"role": "user", "content": message})

    input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt").to(model.device)

    with torch.no_grad():
        output = model.generate(
            input_ids,
            max_new_tokens=1024,
            do_sample=True,
            top_p=0.9,
            temperature=0.7,
            repetition_penalty=1.1,
            eos_token_id=tokenizer.eos_token_id,
        )

    response = tokenizer.decode(output[0][input_ids.shape[-1]:], skip_special_tokens=True)
    history.append((message, response))
    return "", history

# Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("# 🤖 Qwen 2.5 7B Instruct Chatbot")
    chatbot = gr.Chatbot(label="Qwen 2.5 7B Chat")
    msg = gr.Textbox(label="Deine Nachricht", placeholder="Frag mich etwas...", lines=2)
    clear = gr.Button("🧹 Verlauf löschen")
    state = gr.State([])

    msg.submit(generate_response, [msg, state], [msg, chatbot])
    clear.click(lambda: ([], ""), None, [chatbot, state])

demo.launch()

ERROR: Invalid requirement: 'gradio,': Expected end or semicolon (after name and no valid version specifier)
    gradio,
          ^


ModuleNotFoundError: No module named 'gradio'